# Semantic Style GAN のモーフィング生成

## 1. 事前学習済みモデルをダウンロードする
学習済みモデルをpklファイルをダウンロードして，`pretrained/` に配置する．**DL : https://github.com/seasonSH/SemanticStyleGAN/releases/download/1.0.0/CelebAMask-HQ-512x512.pt**

## 2. 画像のサイズを事前学習済みモデルに合わせる
CelebAMask-HQでは512x512サイズであるので，入力画像もそのサイズにリサイズする．`images/`を作成して，入力画像を配置して以下のプログラムを動かす．元画像が上書きされるので注意が必要．

In [ ]:
!python resize512.py --indir images

## 3. 画像をモデルにinvertする（学習済みモデルに則して潜在変数を探索してファインチューニングをする．）
`images/`にある顔画像を全通りinvertする．従って，ある顔画像でinvertして，それでファインチューニングしたモデルを用いて別の顔画像で更にファインチューニングをする．顔画像は順不同なので${}_n C_2$個のファインチューニング済みモデルが作成される．潜在変数は`results/inversion/latent/*-*/`，ファインチューニング済みモデルは`/pretrained`に出力される．やり直すときは`/pretrained`のptファイルを削除しておくこと．

### 3.1. 学習済みモデルを用いて全ての顔画像に対してinvertする
作成されたファインチューニング済みモデルは`/results/inversion/weights`に保存される．

In [3]:
!PYTHONPATH=.:$PYTHONPATH python visualize/invert.py --ckpt pretrained/CelebAMask-HQ-512x512.pt --imgdir images --outdir results/inversion --size 512 --finetune_step 300

Namespace(batch_size=1, ckpt='pretrained/CelebAMask-HQ-512x512.pt', finetune_step=300, imgdir='images', lambda_lpips=1.0, lambda_mean=1.0, lambda_mse=0.1, lr=0.1, lr_g=0.0001, no_noises=True, noise_regularize=10, original_ckpt_path='pretrained/CelebAMask-HQ-512x512.pt', outdir='results/inversion', save_steps=False, size=512, step=400, truncation=1, w_plus=True)
Loading model ...
n_latent: 28, n_latent_expand: 130
model: CelebAMask-HQ-512x512, image: SA
perc: 0.1202 noise: 1.0001 mse: 0.0164  latent: 0.0329: 100%|#| 400/400 [01:49<0
perc: 0.0259 mse: 0.0024: 100%|###############| 300/300 [01:40<00:00,  2.98it/s]


### (3.2.) 潜在変数を確認する
潜在変数が本当に指定した顔に近づいているか，顔表情，マスク，スペクトルをを確認する．

In [4]:
!python components_display.py

Display SA...
Loading model ...
n_latent: 28, n_latent_expand: 130
Generating images...


### 3.3. ファインチューニングした全てのモデルを用いて全ての顔画像に対してinvertする

In [7]:
!python pair_inverting.py

Inverting from SA.pt ...
Namespace(batch_size=1, ckpt='results/inversion/weights/SA.pt', finetune_step=300, imgdir='images', lambda_lpips=1.0, lambda_mean=1.0, lambda_mse=0.1, lr=0.1, lr_g=0.0001, no_noises=True, noise_regularize=10, original_ckpt_path='pretrained/CelebAMask-HQ-512x512.pt', outdir='results/inversion', save_steps=False, size=512, step=400, truncation=1, w_plus=True)
Loading model ...
n_latent: 28, n_latent_expand: 130
model: SA, image: HA
perc: 0.0937 noise: 1.0001 mse: 0.0086  latent: 0.0414: 100%|#| 400/400 [01:56<0
perc: 0.0243 mse: 0.0023: 100%|###############| 300/300 [01:46<00:00,  2.81it/s]
model: SA, image: SA
perc: 0.0295 noise: 1.0001 mse: 0.0027  latent: 0.0260: 100%|#| 400/400 [01:56<0
model: SA, image: SU
perc: 0.0607 noise: 1.0001 mse: 0.0101  latent: 0.0296: 100%|#| 400/400 [01:56<0
perc: 0.0159 mse: 0.0015: 100%|###############| 300/300 [01:45<00:00,  2.84it/s]


## 4. モーフィングを作成する
`/pretrained`のモデル順にモーフィングを作成する．逆順のモーフィングは作成されないのが逆再生で対応できる．顔パーツ区間に分けながら潜在変数で線形モーフィングする．線形モーフィングは $ I^M=\alpha I^S+(1.0-\alpha) I^T \quad(0 \leq \alpha \leq 1.0)$ ではあるが，$ I^S $，$ I^T $は単に画素値ではなく潜在変数であるこに留意する．

In [8]:
!python morphing.py

Morphing SA-HA
Loading model ...
n_latent: 28, n_latent_expand: 130
Generating original image ...
Generating morphing SA -> HA
100%|#########################################| 100/100 [00:52<00:00,  1.92it/s]
Morphing SA-SU
Loading model ...
n_latent: 28, n_latent_expand: 130
Generating original image ...
Generating morphing SA -> SU
100%|#########################################| 100/100 [00:40<00:00,  2.44it/s]
